<a href="https://colab.research.google.com/github/naufalhisyam/TurbidityPrediction-thesis/blob/main/train_model_DenseNet121_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
!pip install tensorflow-addons
import tensorflow_addons as tfa
from sklearn.model_selection import KFold
%load_ext tensorboard

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 14.3 MB/s 


In [2]:
!git clone https://github.com/naufalhisyam/TurbidityPrediction-thesis.git
os.chdir('/content/TurbidityPrediction-thesis') 

Cloning into 'TurbidityPrediction-thesis'...
remote: Enumerating objects: 2998, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 2998 (delta 2), reused 0 (delta 0), pack-reused 2990
Receiving objects: 100% (2998/2998), 670.92 MiB | 31.20 MiB/s, done.
Resolving deltas: 100% (131/131), done.
Checking out files: 100% (2866/2866), done.


**PREPARING DATASET**

In [3]:
images = pd.read_csv(r'./Datasets/0degree/0degInfo.csv') #load dataset info
Y = images[['Turbidity']]

In [4]:
def get_model():
    #Create model
    base_model = tf.keras.applications.DenseNet121(
        include_top=False, weights='imagenet', 
        input_shape=(224, 224, 3), pooling='avg')
    out = base_model.output
    prediction = tf.keras.layers.Dense(1, activation="linear")(out)
    model = tf.keras.Model(inputs = base_model.input, outputs = prediction)

    #Compile the model
    
    return model

tf.test.gpu_device_name()

'/device:GPU:0'

In [5]:
kf = KFold(n_splits = 5)
def get_model_name(k):
    return 'model_'+str(k)+'.h5'

In [6]:
idg = tf.keras.preprocessing.image.ImageDataGenerator(horizontal_flip=True)

In [7]:
VALIDATION_R2 = []
VALIDAITON_LOSS = []

save_dir = '/saved_models/'
fold_var = 1

In [8]:
for train_index, val_index in kf.split(np.zeros(Y.shape[0]),Y):
  training_data = images.iloc[train_index]
  validation_data = images.iloc[val_index]
	
  train_data_generator = idg.flow_from_dataframe(training_data,
                                                 x_col = "Filepath", y_col = "Turbidity",
                                                 target_size=(224, 224), color_mode='rgb',
                                                 class_mode = "raw", shuffle =False)
  valid_data_generator  = idg.flow_from_dataframe(validation_data,
                                                  x_col = "Filepath", y_col = "Turbidity",
                                                  target_size=(224, 224), color_mode='rgb',
                                                  class_mode = "raw", shuffle = False)
	
	# CREATE NEW MODEL
  model = get_model()
	# COMPILE NEW MODEL
  opt = tf.keras.optimizers.Adam(learning_rate=1e-4)
  model.compile(loss=tf.keras.losses.Huber(), optimizer=opt, metrics=[tfa.metrics.RSquare(name="R2")])
	
	# CREATE CALLBACKS
  checkpoint = tf.keras.callbacks.ModelCheckpoint(save_dir+get_model_name(fold_var), monitor='val_loss', verbose=1, save_best_only=True, mode='min')
  callbacks_list = [checkpoint]
	# There can be other callbacks, but just showing one because it involves the model name
	# This saves the best model
	# FIT THE MODEL
  history = model.fit(train_data_generator,
                      epochs=50, batch_size=8,
                      callbacks=callbacks_list,
                      validation_data=valid_data_generator)
	#PLOT HISTORY
	#		:
	#		:
	
	# LOAD BEST MODEL to evaluate the performance of the model
  model.load_weights("/saved_models/model_"+str(fold_var)+".h5")
	
  results = model.evaluate(valid_data_generator)
  results = dict(zip(model.metrics_names,results))
	
  VALIDATION_R2.append(results['R2'])
  VALIDATION_LOSS.append(results['loss'])
	
  tf.keras.backend.clear_session()
	
  fold_var += 1

Found 616 validated image filenames.
Found 155 validated image filenames.
29097984/29084464 [==============================] - 0s 0us/step
Epoch 1/50
20/20 [==============================] - ETA: 0s - loss: 90.5993 - R2: -2.4951
Epoch 1: val_loss improved from inf to 89.67284, saving model to /saved_models/model_1.h5
20/20 [==============================] - 53s 720ms/step - loss: 90.5993 - R2: -2.4951 - val_loss: 89.6728 - val_R2: -2.7249
Epoch 2/50
20/20 [==============================] - ETA: 0s - loss: 87.6742 - R2: -2.2728
Epoch 2: val_loss improved from 89.67284 to 88.77078, saving model to /saved_models/model_1.h5
20/20 [==============================] - 9s 431ms/step - loss: 87.6742 - R2: -2.2728 - val_loss: 88.7708 - val_R2: -2.6060
Epoch 3/50
20/20 [==============================] - ETA: 0s - loss: 85.2649 - R2: -2.1310
Epoch 3: val_loss improved from 88.77078 to 85.57671, saving model to /saved_models/model_1.h5
20/20 [==============================] - 9s 443ms/step - loss: 8

NameError: ignored

**CREATING THE MODEL**

Model Architecture

In [ ]:
pred_turbid = np.squeeze(model.predict(test_images))
true_turbid = test_images.labels
residuals = true_turbid - pred_turbid

In [ ]:
f, axs = plt.subplots(1, 2, figsize=(8,5), gridspec_kw={'width_ratios': [4, 1]})

axs[0].scatter(pred_turbid,residuals)
axs[0].set_title('Residual Plot dari Model DenseNet-121', fontsize=13, fontweight='bold')           
axs[0].set_ylabel('Residual')
axs[0].set_xlabel('Predicted Turbidity')      
axs[0].axhline(0, color='black')
axs[0].grid()

axs[1].hist(residuals, bins=40, orientation="horizontal", density=True)
axs[1].axhline(0, color='black')
axs[1].set_xlabel('Distribution')  
axs[1].yaxis.tick_right()
axs[1].grid(axis='y')

plt.subplots_adjust(wspace=0.05)
plt.savefig(f'saved_model/{name}/residualPlot_{name}.png', dpi=150)
plt.show()

In [ ]:
ms_error = history.history['loss']
val_ms_error = history.history['val_loss']
ma_error = history.history['mae']
val_ma_error = history.history['val_mae']
r2 = history.history['R2']
val_r2 = history.history['val_R2']

epochs = range(1, len(ms_error) + 1)

f, axs = plt.subplots(3, 1, figsize=(6,14))
axs[0].plot(epochs, ms_error, 'tab:orange', label='train_loss (mse)')
axs[0].plot(epochs, val_ms_error, 'tab:blue', label='val_loss (mse)')
axs[0].set_title('MSE Selama Training', fontsize=13, fontweight='bold')
axs[0].set_xlabel('Epoch')
axs[0].set_ylabel('MSE')
axs[0].legend(facecolor='white')
axs[0].grid()

axs[1].plot(epochs, ma_error, 'tab:orange', label='train_mae')
axs[1].plot(epochs, val_ma_error, 'tab:blue', label='val_mae')
axs[1].set_title('MAE Selama Training', fontsize=13, fontweight='bold')
axs[1].set_xlabel('Epoch')
axs[1].set_ylabel('MAE')
axs[1].legend(facecolor='white')
axs[1].grid()

axs[2].plot(epochs, r2, 'tab:orange', label='train_R2')
axs[2].plot(epochs, val_r2, 'tab:blue', label='val_R2')
axs[2].set_title('$R^2$ Selama Training', fontsize=13, fontweight='bold')
axs[2].set_xlabel('Epoch')
axs[2].set_ylabel('$R^2$')
axs[2].legend(facecolor='white')
axs[2].grid()

plt.tight_layout()
plt.savefig(f'saved_model/{name}/trainPlot_{name}.png', dpi=150)
plt.show()

Copy to Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

model_name = 'DenseNet_90deg_imagenet_lr1e-4_decay1e-3_bs8_huber'

save_path = f"/content/gdrive/MyDrive/Hasil_Training/DenseNet/{model_name}"
if not os.path.exists(save_path):
  os.makedirs(save_path)

oripath = "saved_model/."
!cp -a "{oripath}" "{save_path}" # copies files to google drive